In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [7]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [8]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [9]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6777,69.238544,0.444308,15.657414,1.076381,2.734591,0.083319,1.280502,2.615109,3.454852,3.414979
6778,71.099701,0.347868,15.201404,1.102246,2.202415,0.073401,0.874249,1.877795,3.525642,3.695279
6779,71.097912,0.347953,15.202072,1.101415,2.203376,0.073409,0.874700,1.878772,3.525486,3.694906
6780,71.096734,0.348005,15.202582,1.100452,2.204239,0.073413,0.875043,1.879572,3.525337,3.694623
6781,71.096217,0.348023,15.202918,1.099378,2.204977,0.073412,0.875267,1.880171,3.525198,3.694440


In [10]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6777,69.238544,0.444308,15.657414,1.076381,2.734591,0.083319,1.280502,2.615109,3.454852,3.414979
6778,71.099701,0.347868,15.201404,1.102246,2.202415,0.073401,0.874249,1.877795,3.525642,3.695279
6779,71.097912,0.347953,15.202072,1.101415,2.203376,0.073409,0.874700,1.878772,3.525486,3.694906
6780,71.096734,0.348005,15.202582,1.100452,2.204239,0.073413,0.875043,1.879572,3.525337,3.694623
6781,71.096217,0.348023,15.202918,1.099378,2.204977,0.073412,0.875267,1.880171,3.525198,3.694440
...,...,...,...,...,...,...,...,...,...,...
261339,67.628129,0.536188,15.917756,1.293001,3.035754,0.092261,1.656299,3.202303,3.414994,3.223315
261340,67.627570,0.536244,15.917777,1.293965,3.035349,0.092268,1.656338,3.202130,3.415063,3.223296
261341,67.626474,0.536333,15.917866,1.295084,3.034995,0.092279,1.656500,3.202121,3.415126,3.223222
261344,67.748330,0.531097,15.895748,1.355080,2.969883,0.091957,1.617793,3.123769,3.425514,3.240830


In [11]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:950000]

In [12]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2/area2_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2/area2_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [16]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2/area2_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2/area2_2_results_excel.txt")

In [17]:
df_final = pd.concat([df_final1, df_final2])

In [18]:
df_final

,Q,P,K
SAMPLE,,,
6777,26.940096,39.281659,23.102748
6778,29.892830,36.163098,24.818975
6779,29.889517,36.166771,24.816662
6780,29.887060,36.169596,24.814820
6781,29.885414,36.171505,24.813638
...,...,...,...
261339,24.367887,41.891039,21.936420
261340,24.367991,41.890731,21.936420
261341,24.367348,41.891213,21.936008


## QAPF classification

In [19]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [20]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [21]:
df_final.to_excel("../_CIPW/CIPW/AREA2/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2/QAPF_counts.xlsx")

-----

In [25]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=[0], usecols = [0, 1, 2, 3, 4, 29])

In [26]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,7.880625e+05,4.936585e+06,53.0,T,2.0,52.526539,17.979983,22.933498,monzo granite
2,-5.134927e+05,6.119173e+06,53.0,T,1.0,45.821201,19.425633,32.374005,monzo granite
3,1.544245e+06,7.110278e+06,53.0,T,3.0,50.716606,6.475010,37.375668,syeno granite
4,1.083775e+06,6.877413e+06,53.0,T,3.0,41.397182,23.234657,27.592905,monzo granite
5,1.641085e+06,7.925964e+06,53.0,T,4.0,54.609970,18.013528,19.746868,monzo granite
...,...,...,...,...,...,...,...,...,...
4654,6.202597e+05,5.148324e+06,53.0,T,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
4655,1.756745e+06,7.702655e+06,53.0,T,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
4656,6.586139e+05,5.200781e+06,53.0,T,2.0,0.000000,52.355599,12.039131,monzodiorite monzogabbro
4658,1.177291e+06,6.811066e+06,53.0,T,3.0,0.000000,71.436466,10.147088,monzodiorite monzogabbro


In [27]:
QAPF_control_area2 = QAPF_control[QAPF_control['area'] ==2]
QAPF_control_area2.to_excel("../_CIPW/CIPW/AREA2/QAPF_control_points.xlsx", index=True)

In [28]:
QAPF_control_area2

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,788062.538051,4.936585e+06,53.0,T,2.0,52.526539,17.979983,22.933498,monzo granite
7,609204.390926,5.261750e+06,53.0,T,2.0,43.913578,28.900560,23.948371,monzo granite
8,683161.686140,5.345172e+06,53.0,T,2.0,55.002583,18.817920,19.755253,monzo granite
13,566277.769165,5.115580e+06,53.0,T,2.0,42.445293,29.966574,23.523890,monzo granite
16,513561.171031,5.820370e+06,53.0,T,2.0,40.720371,33.121540,20.140040,monzo granite
...,...,...,...,...,...,...,...,...,...
4646,626698.329397,4.987292e+06,53.0,T,2.0,2.711446,54.229187,8.713380,monzodiorite monzogabbro
4649,461509.372701,5.662951e+06,53.0,T,2.0,3.945260,59.519575,6.028671,quartz diorite\nquartz gabbro\nquartz anorthosite
4650,898558.158803,5.868297e+06,53.0,T,2.0,0.707287,53.989234,14.461989,monzodiorite monzogabbro
4654,620259.749413,5.148324e+06,53.0,T,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
